In [1]:
    import importnb, importlib
    from importnb import Notebook, Parameterize
    import IPython, textwrap
    with __import__('importnb').Notebook():
        try: from . import tangle
        except: import tangle

In [2]:
    class PidginMixin:
        extensions = '.md.ipynb',
        def code(self, str):
            transform_cell = IPython.core.inputtransformer2.TransformerManager().transform_cell
            statements = ''
            for statement in (
                transform_cell(object) .rstrip() for object in tangle.markdown_to_statements(str)
            ):
                statements += statement.rstrip('\\')
                if not statement.endswith('\\'): statements += ';'
                statements += ' '
            
            source = transform_cell(tangle.markdown_to_python(str))
            end = len(source.rstrip().rstrip(';'))
            
            code = source[:end]
            if code.strip(): code += '; '
                
            if statements.strip():  
                code += statements
            code +=  source[end:].lstrip(';')
            return code

In [27]:
    class PidginImporter(PidginMixin, Notebook): ...

`MarkdownImporter` imports __Markdown__ files are source.  By default they recieve a Markdown repr.

In [28]:
    class MarkdownMixin:
        extensions = '.md',
        def get_data(self, path): return self.code(self.decode())
        def exec_module(self, module):
            super().exec_module(module)
            module._ipython_display_ = lambda: IPython.display.display(
                IPython.display.Markdown(filename=module.__file__))

In [29]:
    class MarkdownImporter(MarkdownMixin, PidginMixin, Notebook):  ...

In [30]:
    class MarkdownParameterize(MarkdownMixin, PidginMixin, Parameterize):  ...

In [31]:
    class PidginParameterize(PidginMixin, Parameterize): ...

In [32]:
    if importnb.ipython_extension.IPYTHON_MAIN():
        Pidgin, Markdown = PidginParameterize, MarkdownParameterize
    else:
        Pidgin, Markdown = PidginImporter, MarkdownImporter

In [33]:
    def load_ipython_extension(ip=None):         
        Pidgin(position=-1).__enter__()
        if Markdown: Markdown(position=-1).__enter__()

    def unload_ipython_extension(ip): 
        Pidgin().__exit__()
        if Markdown: Markdown().__exit__()

### if __name__ == '__main__': 
        __import__('unittest').TextTestRunner().run(Test())